Code for using VGG16 network as quality metric for our model

discriminator

In [ ]:

from keras.applications.vgg16 import VGG16
vgg_disc = VGG16(include_top=0,input_shape=(256,256,3),classes=1000) # real, fake
print(vgg_disc.summary())


create discriminator

In [ ]:
vgg_out_shape = (8,8,512)
​
for layer in vgg_disc.layers[:]:
    layer.trainable = False
    
discriminator = keras.Sequential()
​
# Add the vgg convolutional base model
discriminator.add(vgg_disc)
# Add new layers
discriminator.add(keras.layers.Flatten())
discriminator.add(Dense(128, activation='relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(2, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
discriminator.summary()
​
discriminator.compile(loss='mae',
              optimizer=keras.optimizers.Adam(lr=1e-6),
              metrics=['acc'])


test initialization

In [ ]:
from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
im = tf.reshape(preprocess_input(train_image[0]),(-1,256,256,3))
​
yhat = discriminator.predict(im)
print(yhat)

training of discriminator and data generation

In [ ]:
print(len(mask_array_edit))

gen_images = np.zeros((len(mask_array_edit),SIZE,SIZE,3))
for i in tqdm(range(len(mask_array_edit))):
    gen_images[i,:,:,:] = np.reshape(model.predict(tf.reshape(mask_array_edit[i],
                                                              (1,SIZE,SIZE,3))),(1,SIZE,SIZE,3))       
​
gen_images = tf.Variable(gen_images)

In [ ]:
train_ims_tf = tf.Variable(np.reshape(train_image,(-1,256,256,3)))
for i in range(10):
    plt.imshow(test_image[i])
    plt.show()
    plt.imshow(gen_images[len(train_image) + i])
    plt.show()
print(tf.shape(gen_images))
print(tf.shape(train_ims_tf)) # list of tensors

disc_data = tf.concat([tf.cast(gen_images,dtype='float32'), tf.cast(train_ims_tf,dtype='float32')],0)
print(tf.shape(disc_data))
nims = tf.shape(gen_images)[0]
nims2 = tf.shape(train_ims_tf)[0]
​
disc_tags = tf.concat([tf.concat([tf.zeros((nims,1),dtype='int32'),tf.ones((nims,1),dtype='int32')],1), 
             tf.concat([tf.ones((nims2,1),dtype='int32'),tf.zeros((nims2,1),dtype='int32')],1)],0)
print(disc_tags[0])
print(disc_tags[1500])
print(tf.shape(disc_tags))

discriminator.fit(disc_data, disc_tags,batch_size=64,epochs=10, validation_split = 0.2,shuffle='True')

Test the discriminator

In [ ]:
preds = []
for i in tqdm(range(tf.shape(gen_images)[0])):
    preds.append(discriminator(tf.reshape(gen_images[i],(-1,256,256,3))))


In [ ]:
preds = np.asarray(preds)
vals = np.argsort(-preds[:,0][:,0])
print(vals[0:4])
​
for iter in range(20):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(gen_images[vals[iter]])
    plt.title(preds[vals[iter]])
    plt.subplot(1,2,2)
    if vals[iter] < len(train_image):
        plt.imshow(train_image[vals[iter]])
        plt.show()